In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import importlib

import itertools as it
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm

from itertools import product
from pandas_profiling import ProfileReport

from criteriaetl.utils.expansion_func import (get_value_counts_with_expansion_factor,
    get_percentage_table_with_expansion_factor)
from criteriaetl.utils.display_func import cdisplay, percentage_count_plot
from criteriaetl.utils.common_func import (get_weighted_complete_randomization_series_on_subset, 
    proportional_cut, weighted_qcut, get_partition_bool_columns_dict)
from criteriaetl.transformers.columns_base import (NameTransformer, 
    ReplaceTransformer, SelectTransformer, AssignTransformer)
from criteriaetl.transformers.rows_base import AggregateTransformer
from criteriaetl.transformers.fusion_base import MergeTransformer

from projectetl.utils.dataload import (load_survey_data_do, save_survey_with_pickle,
                                       load_survey_from_pickle)
from projectetl.utils.config import ENHOGAR_DIR

# Configuration variables

In [ ]:
save_report = False
save_correlation = False

# Load ENHOGAR 2016

In [ ]:
survey_elected_raw = load_survey_data_do(ENHOGAR_DIR / 'Elegidos_ENHOGAR2016_PUB.sav',
                                         load_func=lambda path: pd.read_spss(str(path)))
survey_member_raw = load_survey_data_do(ENHOGAR_DIR / 'Personas_ENHOGAR2016_PUB.sav',
                                        load_func=lambda path: pd.read_spss(str(path)))
survey_house_raw = load_survey_data_do(ENHOGAR_DIR / 'Viviendas_ENHOGAR2016_PUB.sav',
                                       load_func=lambda path: pd.read_spss(str(path)))

# Desired variables

In [ ]:
# hhogar 
# ---
# h402 = sexo
# h403 = edad
# h405 = parentesco
# h407 = estado civil (h407a = números)
# ----
# h409 = aprendió a leer y escribir
# h410 = donde aprendió a leer y escribir
# h411 = asiste o asistió a centro educativo
# h412 = nivel educativo más alto
# h413 = curso o año más alto que cursó
# h414 = asiste en el actual año escolar a algún centro educativo
# h415 = nivel educativo al que está asistiendo
# h416 = curso o año al que está asistiendo
# ---
# h501 = ¿Trabajó o realizó (nombre) alguna actividad económica por lo menos una hora la semana pasada?
# h502 = Aunque no trabajó la semana pasada ¿Tenía algún empleo, negocio o actividad?
# h503 = ¿La semana pasada (nombre) cultivó, cosechó o cuidó ganado u otros animales?
# h504 = ¿La semana pasada, elaboró (nombre) algún producto como artesanía o comida para vender?
# h505 = ¿La semana pasada, ayudó (nombre) a algún familiar en su negocio, empresa o finca?
# h506 = ¿La semana pasada (nombre) cosió, planchó, limpió casas, lavó ropa ajena o realizó otra actividad por paga?
# h507 = Durante las últimas cuatro semanas ¿Ha buscado (nombre) trabajo o estuvo tratando de establecer su propio negocio, actividad económica o empresa?
# h508 = ¿Por qué (nombre) no ha buscado trabajo?
# lh509 = ¿Podría (nombre) haber aceptado un trabajo la semana pasada si le hubieran ofrecido uno?
# lh510 = ¿Dispone (nombre) del tiempo y las condiciones necesarias para salir a trabajar?
# lh511 = La semana pasada, ¿Habría tenido (nombre) el tiempo y las condiciones necesarias para salir a trabajar?
# lh512 = ¿Ha trabajado (nombre) antes por paga o ganancia?
# lh513 = ¿Cuál fue la principal ocupación u oficio línea que desempeñó (nombre) durante la semana pasada o en el último trabajo que tuvo?
# lh514 = ¿A qué se dedica la empresa, negocio o institución en la que trabaja (nombre) actualmente o trabajó por última vez?
# lh515 = ¿En ese trabajo (nombre) es o era...
# lh516 = Regularmente, ¿Cuántas horas a la semana trabaja o trabajó (nombre) en su ocupación principal?
# h601 = ¿Tiene (nombre) PREG. algún seguro de salud, seguro médico o plan de salud?
# region
member_columns = [
    # Identifiers
    'hhupm', 'hhviv', 'hhogar', 'hzona', 'hprovi', 'h402', 'h403', 'h405', 'h407',
    'tipo_hogar',
    # Education
    'h409', 'h410', 'h411', 'h412', 'h413', 'h414',
    'h415', 'h416',
    # Economic characteristics
    'h501', 'h502', 'h503', 'h504', 'h505', 'h506', 
    'h507', 'h508', 'lh509', 'lh510', 'lh511', 'lh512', 'lh513',
    'lh514', 'lh515', 'lh516',
    # Assurance
    'h601',
    # Stratos
    'hestrat', 'region', 'grupsec', 'zona', 'hmiembro',
    'hprovin', 'estratos',
    # Factors
    'fexpansion', 'fponderacion'
]

# h702a = Número de personas estándar mayores de 10 en el hogar
# h704 = edad
# h1001 = ¿Trabajó usted la semana pasada por paga o ganacia?
# h1002 = ¿Cuántos días trabajó usted durante la semana pasada?
# h1003 = ¿Cuántas horas trabajó usted diariamente la semana pasada?
# h1005 = ¿Estudia usted actualmente?
# h1009 = ¿Realiza usted algún tipo de estudio vocacional, técnico o profesional como música, pintura, computación u otra actividad similar?
# h1015 = Durante la semana pasada ¿Dedicó usted algún tiempo al cuidado de enfermos, ancianos, niños u otras personas, ya sean de su hogar u otro hogar?
# h1016 = Durante la semana pasada ¿Cuántos días le dedico al cuidado de esa o esas personas?
# h1017 = Durante la semana pasada ¿Cuántas horas le dedicó usted al cuidado de esa o esas personas diariamente o el último día que cuidó a esa o esas personas?
# h1030 = En la semana pasada, ¿Cuántos días dedicó usted tiempo para criar o cuidar animales o realizar algún cultivo o cosecha de productos para el consumo del hogar?
# h1032 = Durante la semana pasada, ¿Cuántos días dedicó usted a realizar algún trabajo sin paga en un negocio o establecimiento de comercio o producción?
# h1034 = En la semana pasada, ¿Cuántos días dedicó usted tiempo para realizar actividades domésticas en su hogar, como limpiar la casa, cocinar, lavar, fregar u otras labores domésticas?
# h1035 = En la semana pasada, ¿Cuántas horas dedicó usted a realizar actividades diariamente o el último día que las realizó la semana pasada?
# h1036 = En la semana pasada, ¿Cuántos días dedicó usted tiempo para realizar actividades domésticas sin paga en otro hogar, como limpiar la casa, cocinar, lavar, fregar u otras labores domésticas?
# h1037 = Durante la semana pasada, ¿Cuántas horas dedicó usted a realizar esas actividades diariamente o el último día que las ralizó la semana pasada?
elected_columns = [
    'h703', 'h702a',
    'h704', 'h1001', 'h1002', 'h1003', 'h1005',
    'h1009', 'h1030', 'h1032',
    'fexpansion_tic', 'fponderacion_tic',
    # Target
    'h1015', 'h1016', 'h1017', 'h1034', 'h1035',
    'h1036', 'h1037'
]

# Processing surveys

In [ ]:
basic_cols = [
    'hhupm', 'hhviv', 'hhogar'
]

## Assign Transformer

In [ ]:
assign_member_map = {
    'temp': lambda _: 1,
    'h703': lambda df, cols=basic_cols: df.groupby(cols)['temp'].transform('cumsum'),

    'es_jefe': lambda df: df['h405'] == 'El (la) Jefe(a)',
    'es_hijo': lambda df: df['h405'] == 'Hijo(a)',
    'es_conyugue': lambda df: df['h405'] == 'Esposo(a) o compañero(a)',
    'otro_pariente': lambda df: df['h405'].isin(['Yerno o nuera', 'Nieto(a)', 'Padre o madre',
                                                       'Suegro(a)', 'Hermano(a)', 'Cuñado(a)', 'Tío(a)',
                                                       'Sobrino(a)', 'Otro pariente', 'Adoptado/ hijo de crianza']),
    'no_pariente': lambda df: df['h405'].isin(['Trabajador(a) doméstico(a)', 'Otro']),

    'es_jefe_sum': lambda df, cols=basic_cols: df.groupby(cols)['es_jefe'].transform('sum'),
    'es_hijo_sum': lambda df, cols=basic_cols: df.groupby(cols)['es_hijo'].transform('sum'),
    'es_conyugue_sum': lambda df, cols=basic_cols: df.groupby(cols)['es_conyugue'].transform('sum'),
    'otro_pariente_sum': lambda df, cols=basic_cols: df.groupby(cols)['otro_pariente'].transform('sum'),
    'no_pariente_sum': lambda df, cols=basic_cols: df.groupby(cols)['no_pariente'].transform('sum'),

    'tipo_hogar': lambda df:
        np.select([
            (lambda df: (df['es_jefe_sum'] == 1) \
                & (df['es_conyugue_sum'] == 0) & (df['es_hijo_sum'] == 0) \
                & (df['otro_pariente_sum'] == 0) & (df['no_pariente_sum'] == 0))(df),
            (lambda df: (df['es_jefe_sum'] == 1) \
                & (df['es_conyugue_sum'] != 0) & (df['es_hijo_sum'] == 0) \
                & (df['otro_pariente_sum'] == 0) & (df['no_pariente_sum'] == 0))(df),
            (lambda df: (df['es_jefe_sum'] == 1) \
                & (df['es_conyugue_sum'] != 0) & (df['es_hijo_sum'] != 0) \
                & (df['otro_pariente_sum'] == 0) & (df['no_pariente_sum'] == 0))(df),
            (lambda df: (df['es_jefe_sum'] == 1) \
                & (df['es_conyugue_sum'] == 0) & (df['es_hijo_sum'] != 0) \
                & (df['otro_pariente_sum'] == 0) & (df['no_pariente_sum'] == 0))(df),
            (lambda df: (df['es_jefe_sum'] == 1) & (df['es_conyugue_sum'] != 0) \
                & (df['es_hijo_sum'] == 0) & (df['otro_pariente_sum'] != 0) \
                & (df['no_pariente_sum'] == 0))(df),
            (lambda df: (df['es_jefe_sum'] == 1) & (df['es_conyugue_sum'] != 0) \
                & (df['es_hijo_sum'] != 0) & (df['otro_pariente_sum'] != 0) \
                & (df['no_pariente_sum'] == 0))(df),
            (lambda df: (df['es_jefe_sum'] == 1) \
                & (df['es_conyugue_sum'] == 0) & (df['es_hijo_sum'] != 0) \
                & (df['otro_pariente_sum'] != 0) & (df['no_pariente_sum'] == 0))(df),
            (lambda df: (df['es_jefe_sum'] == 1) \
                & (df['es_conyugue_sum'] == 0) & (df['es_hijo_sum'] == 0) \
                & (df['otro_pariente_sum'] != 0) & (df['no_pariente_sum'] == 0))(df),
            (lambda df: (df['es_jefe_sum'] == 1) & (df['no_pariente_sum'] != 0))(df)
        ], [
            "Unipersonal", "Nuclear sin hijos", "Nuclear con hijos", "Nuclear monoparental",
            "Extendido base nuclear sin hijos", "Extendido base nuclear con hijos",
            "Extendido base monoparental", "Extendido sin base nuclear", "Compuesto"
        ])
}

assign_member_transformer = AssignTransformer(assign_member_map)
survey_member_assigned = assign_member_transformer.transform(survey_member_raw)

In [ ]:
survey_member_assigned.head()

## Merge Transformer

In [ ]:
merge_member_elected_transformer = MergeTransformer(lambda : survey_member_assigned,
                                                    merge_kwargs={
                                                        'on': basic_cols + ['h703'],
                                                        'suffixes': ('', '_other')
                                                    })
survey_elected_merged = merge_member_elected_transformer.transform(survey_elected_raw)

In [ ]:
survey_elected_merged = survey_elected_merged[member_columns + elected_columns]

## Name Transformer

In [ ]:
name_elected_map = {
    'h402': 'sexo', # sexo
    'h403': 'edad', # edad
    'h405': 'parentesco', # parentesco
    'h407': 'estado_civil',
    'h409': 'aprendio_leer_escribir',
    'h410': 'donde_aprendio_leer_escribir',
    'h411': 'asiste_asistio_centro_educativo',
    'h412': 'nivel_educativo_mas_alto',
    'h413': 'curso_mas_alto_que_curso',
    'h414': 'asiste_centro_educativo_actualmente',
    'h415': 'nivel_esta_asistiendo',
    'h416': 'curso_esta_asistiendo',
    'h501': 'trabajo_semana_pasada',
    'h502': 'tenia_empleo_semana_pasada',
    'h503': 'cultivo_cosecho_semana_pasada',
    'h504': 'elaboro_producto_semana_pasada',
    'h505': 'ayudo_familiar_semana_pasada',
    'h506': 'tuvo_trabajo_domestico_por_paga_semana_pasada',
    'h507': 'busco_trabajo_cuatro_semanas_pasadas',
    'h508': 'motivo_no_busca_trabajo',
    'lh509': 'podria_aceptar_trabajo_semana_pasada',
    'lh510': 'dispone_condiciones_para_trabajar',
    'lh511': 'dispone_condiciones_para_trabajar_semana_pasada',
    'lh512': 'ha_trabajado_por_paga',
    'lh513': 'ultima_principal_ocupacion',
    'lh514': 'se_dedica_ultima_empresa',
    'lh515': 'ocupacion_ultimo_trabajo',
    'lh516': 'horas_trabajadas_semanalmente_ocupacion_principal',
    'h601': 'tiene_seguro_salud_pension',
    'h702a': 'mayores_10_annos',
    'h704': 'edad_elegido',
    'h1001': 'trabajo_por_paga_semana_pasada',
    'h1002': 'dias_trabajo_semana_pasada',
    'h1003': 'horas_trabajo_semana_pasada',
    'h1005': 'estudia_actualmente',
    'h1009': 'realiza_actividad_vocacional',
    'h1015': 'dedico_tiempo_cuidados',
    'h1016': 'tiempo_cuidados_dias',
    'h1017': 'tiempo_cuidado_horas_diarias',
    'h1030': 'tiempo_agricultura_ganado_dias',
    'h1032': 'tiempo_trabajo_sin_paga_dias',
    'h1034': 'tiempo_trabajo_domestico_hogar_dias',
    'h1035': 'tiempo_trabajo_domestico_hogar_horas',
    'h1036': 'tiempo_trabajo_domestico_no_hogar_dias',
    'h1037': 'tiempo_trabajo_domestico_no_hogar_horas'
}

keep = ['hhupm', 'hhviv', 'hhogar', 'hzona', 'hprovi',
        'tipo_hogar',
        # Stratos
        'hestrat', 'region', 'grupsec', 'zona', 'hmiembro',
        'hprovin', 'estratos',
        # Factors
        'fexpansion', 'fponderacion',
        'fexpansion_tic', 'fponderacion_tic',]

name_elected_transformer = NameTransformer(name_elected_map, keep_features=keep)
survey_elected_named = name_elected_transformer.transform(survey_elected_merged)

In [ ]:
categories = set(survey_elected_named.columns) - set(survey_elected_named.select_dtypes(float))
categories

## Replace Transformer

In [ ]:
motivos_no_busca_trabajo = [
    'Ha buscado trabajo y no encuentra', 'Familiar y otras personas le están buscando',
    'Solicitó y espera respuesta', 'Está incapacitado permanentemente',
    'Está Incapacitado temporalmente', 'Está estudiando', 'Se dedica a quehaceres del hogar',
    'Es rentista', 'No tiene suficiente educación o experiencia', 'Cree que no va a encontrar',
    'Por razones de edad', 'No quiso buscar trabajo', 'Pensionado o jubilado'
]

In [ ]:
replace_elected_map = {
    'horas_trabajadas_semanalmente_ocupacion_principal': {
        'No sabe': 0.0
    },
    'curso_mas_alto_que_curso': {
        'No sabe': 0.0
    },
    'curso_esta_asistiendo': {
        'No sabe': 0.0
    },
    'motivo_no_busca_trabajo': {
        **{reason: i + 1 for i, reason in enumerate(motivos_no_busca_trabajo)},
        'Otro': 98
    }
}

astype_elected_map = {
    'horas_trabajadas_semanalmente_ocupacion_principal': float,
    'tiempo_agricultura_ganado_dias': float,
    'tiempo_trabajo_sin_paga_dias': float,
    'curso_mas_alto_que_curso': float,
    'curso_esta_asistiendo': float,
    'motivo_no_busca_trabajo': 'category'
}

replace_elected_transformer = ReplaceTransformer(replace_elected_map, astype_dict=astype_elected_map)
survey_elected_replaced = replace_elected_transformer.transform(survey_elected_named)

## Select Transformer

In [ ]:
age_groups = [(10, 14), (15, 24), (25, 45), (46, 59), (60, 74), (75, None)]

In [ ]:
select_elected_map = {
    'grupo_etareo': {
        lambda df: df['edad'] == '97+': '75-+',
        **{
            lambda df, min_group=min_group, max_group=max_group:
                pd.to_numeric(df['edad'], errors='coerce').isin(range(min_group, (max_group or 96) + 1)):
                lambda _, min_group=min_group, max_group=max_group: f'{min_group}-{max_group or "+"}'
            for min_group, max_group in age_groups
        },
        'default': lambda df: df['edad']
    },

    'condicion_actividad': {
        # ocupados: 
        # trabajo_semana_pasada - realizo algun trabajo la semana anterior por lo menos una hora: sí, o
        # tuvo_trabajo_domestico_por_paga_semana_pasada - en la semana anterior: realizo alguna actividad para 
        # obtener dinero o especies, o,
        # tenia_empleo_semana_pasada - tiene un negocio o empresa a la que proximamente volverá: sí
        lambda df: (df['trabajo_semana_pasada'] == 'Sí') \
                    | (df['tuvo_trabajo_domestico_por_paga_semana_pasada'] == 'Sí') \
                    | (df['tenia_empleo_semana_pasada'] == 'Sí'): 'ocupado',
        
        lambda df: df['motivo_no_busca_trabajo'].isin([4, 5, 11]): 'incapacitado o anciano',
        lambda df: df['motivo_no_busca_trabajo'].isin([8, 13]): 'jubilado o rentista',
        lambda df: (df['motivo_no_busca_trabajo'] == 7): 'quehacer doméstico',
        lambda df: (df['motivo_no_busca_trabajo'] == 6): 'estudiante',

        # desocupados
        # busco_trabajo_cuatro_semanas_pasadas - en las 4 semanas anteriores estuvo buscando trabajo o 
        # tratando de emprender: si, y
        # motivo_no_busca_trabajo = 1, 2, 3
        # r408 - en las cuantro semanas efectivamente hizo algo para 
        # buscar trabajo o emprender
        lambda df: (df['busco_trabajo_cuatro_semanas_pasadas'] == 'Sí') \
                   | df['motivo_no_busca_trabajo'].isin([1, 2, 3]): 'desocupado',

        # inactivos
        'default': 'inactivo'
    },

    'tiempo_cuidados_horas_semanales': {
        lambda df: (df['dedico_tiempo_cuidados'] == 'No') | (df['dedico_tiempo_cuidados'].isna()): 0.0,
        'default': lambda df: df['tiempo_cuidados_dias'] * df['tiempo_cuidado_horas_diarias']
    },

    'tiempo_trabajo_domestico_hogar_horas_semanales': {
        lambda df: (df['tiempo_trabajo_domestico_hogar_dias'] == '97') \
                    | (df['tiempo_trabajo_domestico_hogar_dias'] == 99.0): 0.0,
        lambda df: df['tiempo_trabajo_domestico_hogar_dias'].isna(): 0.0,
        'default': lambda df: df['tiempo_trabajo_domestico_hogar_dias'].astype(float) * df['tiempo_trabajo_domestico_hogar_horas']
    },

    'tiempo_trabajo_domestico_no_hogar_horas_semanales': {
        lambda df: (df['tiempo_trabajo_domestico_no_hogar_dias'] == '97') \
                    | (df['tiempo_trabajo_domestico_no_hogar_dias'] == 99.0): 0.0,
        lambda df: df['tiempo_trabajo_domestico_no_hogar_dias'].isna(): 0.0,
        'default': lambda df: df['tiempo_trabajo_domestico_no_hogar_dias'].astype(float) * df['tiempo_trabajo_domestico_hogar_horas']
    }
}

select_elected_transformer = SelectTransformer(select_elected_map)
survey_elected_selected = select_elected_transformer.transform(survey_elected_replaced)

In [ ]:
get_percentage_table_with_expansion_factor(survey_elected_selected, 'dedico_tiempo_cuidados', 'fexpansion_tic')

## Assign Transformer

In [ ]:
assign_elected_map = {
    'tiempo_trabajo_doméstico_no_remunerado_horas_semanales':
        lambda df: df['tiempo_trabajo_domestico_hogar_horas_semanales'] + df['tiempo_trabajo_domestico_no_hogar_horas_semanales'],

    'dedico_tiempo_trabajo_doméstico_no_remunerado': lambda df: df['tiempo_trabajo_doméstico_no_remunerado_horas_semanales'] > 0,

    'dedico_tiempo_trabajo_doméstico_no_hogar': lambda df: df['tiempo_trabajo_domestico_no_hogar_horas_semanales'] > 0
}

assign_elected_transformer = AssignTransformer(assign_elected_map)
survey_elected_assigned = assign_elected_transformer.transform(survey_elected_selected)

In [ ]:
survey_elected_assigned['dedico_tiempo_trabajo_doméstico_no_hogar'].value_counts()

In [ ]:
get_percentage_table_with_expansion_factor(survey_elected_assigned, 'dedico_tiempo_trabajo_doméstico_no_hogar', 'fexpansion_tic')

In [ ]:
get_percentage_table_with_expansion_factor(survey_elected_assigned, 'dedico_tiempo_trabajo_doméstico_no_remunerado', 'fexpansion_tic')

## Dummify survey

In [ ]:
survey_elected_dummified = pd.get_dummies(survey_elected_assigned, drop_first=True)
len(survey_elected_dummified.columns)

## Fill NA

In [ ]:
survey_elected_nan_replaced = survey_elected_dummified.fillna(0)

In [ ]:
survey_elected_nan_replaced.head()

# EDA

In [ ]:
survey_elected_nan_replaced.loc[survey_elected_nan_replaced['tiempo_cuidados_horas_semanales'] > 0.0, 'tiempo_cuidados_horas_semanales'] \
                           .value_counts()

## Pandas profiling

In [ ]:
if save_report:
    report = ProfileReport(survey_elected_selected, config_file='config.yml')
    report.to_file('report.html')

## Correlation

In [ ]:
if save_correlation:
    target_cols = [
        'tiempo_cuidados_horas_semanales', 'tiempo_trabajo_domestico_hogar_horas_semanales',
        'tiempo_trabajo_domestico_no_hogar_horas_semanales'
    ]
    groups = set(survey_elected_named.columns) - set(target_cols)
    for col in groups:
        cols = [scol for scol in survey_elected_dummified.columns if col in scol]
        sns_plot = sns.heatmap(survey_elected_dummified[cols + target_cols].corr(), vmin=-1, vmax=1, center=0,
                            cmap=sns.diverging_palette(20, 220, n=200),
                            square=True)
        sns_plot.get_figure().savefig(f'correlations/corr_{col}.png')
        sns_plot.get_figure().clf()

## Extra checks

In [ ]:
(survey_elected_dummified['dedico_tiempo_cuidados_Si']).sum()

# Regression Estimator

In [1]:
from sklearn.linear_model import LassoCV, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

from projectetl.utils.functions import get_real_feature_names

/Users/ohch/opt/anaconda3/envs/criteria-do/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/ohch/opt/anaconda3/envs/criteria-do/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


ModuleNotFoundError: No module named 'projectetl.utils.functions'

In [ ]:
feature_names = [
    'sexo',
    'parentesco',
    'estado_civil',
    'aprendio_leer_escribir',
    'donde_aprendio_leer_escribir',
    'asiste_asistio_centro_educativo',
    'nivel_educativo_mas_alto',
    'curso_mas_alto_que_curso',
    'asiste_centro_educativo_actualmente',
    'nivel_esta_asistiendo',
    'curso_esta_asistiendo',
    'trabajo_semana_pasada',
    'tenia_empleo_semana_pasada',
    'cultivo_cosecho_semana_pasada',
    'elaboro_producto_semana_pasada',
    'ayudo_familiar_semana_pasada',
    'tuvo_trabajo_domestico_por_paga_semana_pasada',
    'busco_trabajo_cuatro_semanas_pasadas',
    'motivo_no_busca_trabajo',
    'podria_aceptar_trabajo_semana_pasada',
    'dispone_condiciones_para_trabajar',
    'dispone_condiciones_para_trabajar_semana_pasada',
    'ha_trabajado_por_paga',
    'ultima_principal_ocupacion',
    'se_dedica_ultima_empresa',
    'ocupacion_ultimo_trabajo',
    'horas_trabajadas_semanalmente_ocupacion_principal',
    'tiene_seguro_salud_pension',
    'mayores_10_annos',
    'trabajo_por_paga_semana_pasada',
    'dias_trabajo_semana_pasada',
    'horas_trabajo_semana_pasada',
    'estudia_actualmente',
    'realiza_actividad_vocacional',
    'tiempo_agricultura_ganado_dias',
    'tiempo_trabajo_sin_paga_dias',
    'grupo_etareo',
    'condicion_actividad',
    'tipo_hogar'
    # Stratos
    'region', 'grupsec', 'zona', 'hmiembro',
    'hprovin', 'estratos'
]

feature_names = get_real_feature_names(survey_elected_dummified, feature_names)

In [ ]:
target_col = 'tiempo_trabajo_doméstico_no_remunerado_horas_semanales'
decision_col = 'dedico_tiempo_trabajo_doméstico_no_remunerado'
weight_col = 'fexpansion_tic'

## Simple estimators

In [ ]:
survey_train, survey_test = train_test_split(survey_elected_nan_replaced)

In [ ]:
X_train = survey_train.loc[survey_train[decision_col], feature_names]
y_train = survey_train.loc[survey_train[decision_col], target_col]
weight_train = survey_train.loc[survey_train[decision_col], weight_col]

In [ ]:
X_test = survey_test.loc[survey_test[decision_col], feature_names]
y_test = survey_test.loc[survey_test[decision_col], target_col]
weight_test = survey_test.loc[survey_test[decision_col], weight_col]

### LassoCV

In [ ]:
estimator = LassoCV(cv=5)
estimator = estimator.fit(X_train, y_train)

In [ ]:
selector = SelectFromModel(estimator, prefit=True)
feature_names[selector.get_support()]

### Lasso (`<decision_col> == True`)

In [ ]:
estimator = Lasso(alpha=0.01, max_iter=100000)
estimator = estimator.fit(X_train, y_train, sample_weight=weight_train)

In [ ]:
estimator.score(X_test, y_test, sample_weight=weight_test)

In [ ]:
selector = SelectFromModel(estimator, prefit=True)
feature_names[selector.get_support()]

### Lasso (all)

In [ ]:
X_all_train = survey_train[feature_names]
y_all_train = survey_train[target_col]
weight_all_train = survey_train[weight_col]

In [ ]:
X_all_test = survey_test[feature_names]
y_all_test = survey_test[target_col]
weight_all_test = survey_test[weight_col]

In [ ]:
estimator_all_lasso = Lasso(alpha=0.1, max_iter=100000)
estimator_all_lasso = estimator_all_lasso.fit(X_all_train, y_all_train, sample_weight=weight_all_train)
estimator_all_lasso.score(X_all_test, y_all_test, sample_weight=weight_all_test)

In [ ]:
selector_all_lasso = SelectFromModel(estimator_all_lasso, prefit=True)
feature_names[selector.get_support()]

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
estimator_random_forest = RandomForestRegressor()
estimator_random_forest = estimator_random_forest.fit(X_train, y_train, sample_weight=weight_train)

In [ ]:
estimator_random_forest.score(X_test, y_test, sample_weight=weight_test)

In [ ]:
tolerance = 1e-2

In [ ]:
feature_names[estimator_random_forest.feature_importances_ > tolerance]

## Cross - Validation

In [ ]:
from sklearn.model_selection import GridSearchCV

from projectetl.utils.estimator import get_estimator

In [ ]:
survey_care = survey_elected_nan_replaced.loc[survey_elected_nan_replaced[decision_col], list(feature_names) + [target_col, weight_col]]

### Lasso

In [ ]:
lasso = Lasso(max_iter=100000)

In [ ]:
alphas = [10**(-i) for i in range(5)]
grid_lasso_kwargs = {
    'param_grid': {'alpha': np.array(alphas)},
    'verbose': 1
}

In [ ]:
estimator_grid_lasso, test_indexes_grid_lasso = get_estimator(lasso,
                                                              X=list(feature_names),
                                                              y=target_col,
                                                              df=survey_care,
                                                              grid_kwargs=grid_lasso_kwargs,
                                                              weight=weight_col)

In [ ]:
# selector_grid_lasso = SelectFromModel(estimator_grid_lasso.best_estimator_, prefit=True)
# feature_names[selector_grid_lasso.get_support()]

In [ ]:
len(estimator_grid_lasso.best_estimator_.coef_ > tolerance)

In [ ]:
len(feature_names)

In [ ]:
X_grid_lasso_test = survey_care.loc[test_indexes_grid_lasso, feature_names].values
y_grid_lasso_test = survey_care.loc[test_indexes_grid_lasso, target_col].values
weight_grid_lasso_test = survey_care.loc[test_indexes_grid_lasso, weight_col].values

estimator_grid_lasso.best_estimator_.score(X_grid_lasso_test,
                                           y_grid_lasso_test,
                                           sample_weight=weight_grid_lasso_test)

In [ ]:
estimator_grid_lasso.best_score_

In [ ]:
estimator_grid_lasso.best_estimator_

### Random Forest Regressor

In [ ]:
random_forest = RandomForestRegressor()

In [ ]:
grid_random_forest_kwargs = {
    'param_grid': {'n_estimators': np.array([10**i for i in range(5)])},
    'verbose': 1
}

In [ ]:
estimator_grid_random_forest, test_indexes_grid_random_forest = get_estimator(random_forest,
                                                                              df=survey_care,
                                                                              X=feature_names,
                                                                              y=target_col,
                                                                              grid_kwargs=grid_random_forest_kwargs,
                                                                              weight=weight_col)

In [ ]:
X_grid_random_forest_test = survey_care.loc[test_indexes_grid_random_forest, feature_names].values
y_grid_random_forest_test = survey_care.loc[test_indexes_grid_random_forest, target_col].values
weight_grid_random_forest_test = survey_care.loc[test_indexes_grid_random_forest, weight_col].values

estimator_grid_random_forest.best_estimator_.score(X_grid_random_forest_test,
                                                   y_grid_random_forest_test,
                                                   sample_weight=weight_grid_random_forest_test)

In [ ]:
estimator_grid_random_forest.best_score_

In [ ]:
estimator_grid_random_forest.best_estimator_

In [ ]:
estimator_grid_random_forest.best_params_

## Conditional Mean Estimator

In [ ]:
from sklearn.metrics import mean_squared_error
from projectetl.utils.estimator import ConditionalMeanEstimator

In [ ]:
groupby_features_1 = [
    'sexo', 'region', 'grupo_etareo', 'ocupacion_ultimo_trabajo'
]
groupby_features_2 = [
    'sexo', 'grupo_etareo', 'zona', 'grupsec',
    'region', 'nivel_educativo_mas_alto',
    'ocupacion_ultimo_trabajo', 'condicion_actividad',
    'estratos'
]
groupby_features_3 = [
    'sexo', 'estratos'
]
groupby_features_4 = [
    'sexo', 'zona', 'nivel_educativo_mas_alto'
]
groupby_features_5 = [
    'sexo'
]
groupby_features_6 = [
    'zona'
]
groupby_features_7 = [
    'sexo', 'zona'
]

groupby_index = 7
groupby_features = locals().get(f'groupby_features_{groupby_index}')
groupby_features = list(get_real_feature_names(survey_care, groupby_features))

### Baseline vs Grid Random Forest Regressor

In [ ]:
estimator_baseline_grid_random_forest = ConditionalMeanEstimator(groupby_features)

In [ ]:
train_indexes_grid_random_forest = list(set(survey_care.index.values) - set(test_indexes_grid_random_forest))
survey_baseline_train_grid_random_forest = survey_care.loc[train_indexes_grid_random_forest]
survey_baseline_test_grid_random_forest = survey_care.loc[test_indexes_grid_random_forest]

estimator_baseline_grid_random_forest = estimator_baseline_grid_random_forest.fit(survey_baseline_train_grid_random_forest[feature_names],
                                                                                  survey_baseline_train_grid_random_forest[target_col],
                                                                                  sample_weight=survey_baseline_train_grid_random_forest[weight_col],
                                                                                  weight_col=weight_col)

In [ ]:
y_pred_grid_random_forest = estimator_baseline_grid_random_forest.predict(survey_baseline_test_grid_random_forest[feature_names])
estimator_baseline_grid_random_forest.score(survey_baseline_test_grid_random_forest[target_col],
                                            y_pred_grid_random_forest,
                                            sample_weight=survey_baseline_test_grid_random_forest[weight_col])

In [ ]:
y_pred = estimator_grid_random_forest.best_estimator_.predict(survey_baseline_test_grid_random_forest[feature_names])
mean_squared_error(survey_baseline_test_grid_random_forest[target_col],
                   y_pred,
                   sample_weight=survey_baseline_test_grid_random_forest[weight_col])

### Baseline vs Grid Lasso Regression

In [ ]:
estimator_baseline_grid_lasso = ConditionalMeanEstimator(groupby_features)

In [ ]:
train_indexes_grid_lasso = list(set(survey_care.index.values) - set(test_indexes_grid_lasso))
survey_baseline_train_grid_lasso = survey_care.loc[train_indexes_grid_lasso]
survey_baseline_test_grid_lasso = survey_care.loc[test_indexes_grid_lasso]

estimator_baseline_grid_lasso = estimator_baseline_grid_lasso.fit(survey_baseline_train_grid_lasso[feature_names],
                                                                  survey_baseline_train_grid_lasso[target_col],
                                                                  sample_weight=survey_baseline_train_grid_lasso[weight_col],
                                                                  weight_col=weight_col)

In [ ]:
y_pred_grid_lasso = estimator_baseline_grid_lasso.predict(survey_baseline_test_grid_lasso[feature_names])
estimator_baseline_grid_lasso.score(survey_baseline_test_grid_lasso[target_col],
                                    y_pred_grid_lasso,
                                    sample_weight=survey_baseline_test_grid_lasso[weight_col])

In [ ]:
y_pred = estimator_grid_lasso.best_estimator_.predict(survey_baseline_test_grid_lasso[feature_names])
mean_squared_error(survey_baseline_test_grid_lasso[target_col],
                   y_pred,
                   sample_weight=survey_baseline_test_grid_lasso[weight_col])

# Classification Estimator

In [ ]:
survey_all = survey_elected_nan_replaced.copy()

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic_lasso = LogisticRegression(penalty='l1',
                                    solver='liblinear',
                                    random_state=6202)

In [ ]:
grid_logistic_lasso_kwargs = {
    'param_grid': {'C': np.array([.01, .1, 1, 10])},
    'verbose': 1,
    'scoring': 'neg_log_loss'
}
target_logistic_col = 'dedico_tiempo_trabajo_doméstico_no_remunerado'

In [ ]:
estimator_logistic_lasso, test_indexes_logistic_lasso = get_estimator(logistic_lasso,
                                                                      df=survey_all,
                                                                      X=list(feature_names),
                                                                      y=target_logistic_col,
                                                                      grid_kwargs=grid_logistic_lasso_kwargs,
                                                                      weight=weight_col)

In [ ]:
estimator_logistic_lasso.best_estimator_

In [ ]:
estimator_logistic_lasso.best_params_

In [ ]:
estimator_logistic_lasso.best_score_

In [ ]:
survey_logistic_lasso_test = survey_all.loc[test_indexes_logistic_lasso]
predicted_logistic_lasso = estimator_logistic_lasso.predict(survey_logistic_lasso_test[feature_names])

In [ ]:
get_percentage_table_with_expansion_factor(survey_logistic_lasso_test, target_logistic_col, weight_col)

In [ ]:
estimator_logistic_lasso.best_estimator_.score(survey_logistic_lasso_test[feature_names],
                                               survey_logistic_lasso_test[target_logistic_col],
                                               sample_weight=survey_logistic_lasso_test[weight_col])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(survey_logistic_lasso_test[target_logistic_col],
                            predicted_logistic_lasso,
                            sample_weight=survey_logistic_lasso_test[weight_col]))

In [ ]:
# selector_all_lasso = SelectFromModel(estimator_logistic_lasso, prefit=True)
# feature_names[selector_all_lasso.get_support()]

In [ ]:
list(survey_elected_selected['condicion_actividad'].unique())

In [ ]:
new_feature_names = [
    # sexo
    'sexo_Mujer',
    # parentesco
    'parentesco_Esposo(a) o compañero(a)',
    # estado_civil
    'estado_civil_Casado(a)',
    'estado_civil_Separado(a) de unión libre',
    'estado_civil_Soltero(a)',
    'estado_civil_Unido(a)',
    
    #! 'asiste_asistio_centro_educativo_Nunca ha asistido',
    # nivel_ultimo_ano_aprobado
    'nivel_educativo_mas_alto_Medio o Secundario',
    # ultimo_ano_aprobado
    'curso_mas_alto_que_curso',
    # curso_matriculado
    'curso_esta_asistiendo',
    # trabajo_semana_pasada
    'trabajo_semana_pasada_Sí',
    # busco_trabajo_establ_negocio
    'busco_trabajo_cuatro_semanas_pasadas_No',
    # motivo_no_busca_trabajo
    #! 'motivo_no_busca_trabajo_6.0',
    #! 'motivo_no_busca_trabajo_7.0',
    # aceptaria_trab_sem_pasada
    'podria_aceptar_trabajo_semana_pasada_Sí',

    #! 'ultima_principal_ocupacion',
    # rama_principal_cod, rama_cesantia_cod
    # 'se_dedica_ultima_empresa',
    # categoria_cesantia, categoria_principal
    'ocupacion_ultimo_trabajo_no profesional que trabaja por cuenta propia?',
    'ocupacion_ultimo_trabajo_trabajador(a) doméstica?',
    # horas_trabaja_semana_principal
    'horas_trabajadas_semanalmente_ocupacion_principal',
    # afiliado_seguro_salud
    'tiene_seguro_salud_pension_Sí',
    # mayores_10_annos (!)
    'mayores_10_annos',

    #! 'dias_trabajo_semana_pasada',

    #! 'horas_trabajo_semana_pasada',
    # asiste_centro_educativo
    'estudia_actualmente_Si',
    # realiza_curso_técnico
    'realiza_actividad_vocacional_Si',

    #! 'tiempo_agricultura_ganado_dias',

    #! 'tiempo_trabajo_sin_paga_dias',
    # grupo_etareo
    'grupo_etareo_15-24',
    'grupo_etareo_25-45',
    'grupo_etareo_60-74',
    # condicion_actividad
    'condicion_actividad_quehacer doméstico',

    #! 'grupsec_Medio bajo',
    #! 'grupsec_Medio y Medio alto',
    #! 'grupsec_Muy bajo',
    # miembro
    'hmiembro',
    # des_provincia, id_provincia
    'hprovin_Distrito Nacional',
    'hprovin_La Altagracia',
    'hprovin_Monte Cristi',
    'hprovin_Peravia',
    'hprovin_San Juan',
    'hprovin_San Pedro de Macorís',
    'hprovin_Santiago',
    # des_estrato, estrato
    'estratos_Grandes Ciudades',
    'estratos_Resto Urbano']

In [ ]:
estimator_logistic_lasso, test_indexes_logistic_lasso = get_estimator(logistic_lasso,
                                                                      df=survey_all,
                                                                      X=list(new_feature_names),
                                                                      y=target_logistic_col,
                                                                      grid_kwargs=grid_logistic_lasso_kwargs,
                                                                      weight=weight_col)

In [ ]:
estimator_logistic_lasso.best_estimator_

In [ ]:
survey_logistic_lasso_test = survey_all.loc[test_indexes_logistic_lasso]
predicted_logistic_lasso = estimator_logistic_lasso.predict(survey_logistic_lasso_test[new_feature_names])
predicted_logistic_lasso

In [ ]:
print(classification_report(survey_logistic_lasso_test[target_logistic_col],
                            predicted_logistic_lasso,
                            sample_weight=survey_logistic_lasso_test[weight_col]))